# Instalación de dependencias


# Configuración de cuenta de usuario


In [1]:
from iol_client import IOLClient

Se recomienda usar el archivo `.env` para colocar claves privadas, pero este espacio tambien puede ser usado como punto de entrada de datos privados. Tener la debida precaución.


In [2]:
import os

user = os.getenv("IOL_USER") or ""
password = os.getenv("IOL_PASS") or ""

In [3]:
client_iol = IOLClient(username=user, password=password)

# Historico anual de un determinado instrumento


In [4]:
from datetime import date
from iol_client.constants import Ajustada, Mercado

In [ ]:
hoy = date.today()
hace_un_anio = hoy.replace(year=hoy.year - 1)
historico = await client_iol.get_titulo_historicos(
    simbolo="GGAL",
    mercado=Mercado.BCBA,
    fecha_desde=hace_un_anio,
    fecha_hasta=hoy,
    ajustada=Ajustada.AJUSTADA,
)

In [5]:
import pandas as pd


In [ ]:

df = pd.DataFrame(data=historico)
df.head()

## Guardando los datos


Exportamos datos a un excel


In [ ]:
# df.to_excel("historico.xlsx")

Leer datos desde excel


In [ ]:
# df = pd.read_excel("historico.xlsx")

## Analisis y visualización


Primera vista de los datos


In [ ]:
df.head()

Cantidad de registros encontrados


In [ ]:
df.shape

Graficando variables en función del tiempo


In [ ]:
df.set_index("fechaHora", inplace=True)
df.head()

In [ ]:
df.sort_index(inplace=True)

In [ ]:
fig = df.plot.line(y="ultimoPrecio", rot=0, figsize=(20, 10), grid=True)
fig.set_xlabel("Fecha")
fig.set_ylabel("Precio(pesos argentinos)")

La realidad es que de los historicos, solo nos interesa el simbolo, la fecha, el valor de apertura, maximo y minimo alcanzado, cierre, volumen/monto y volumen nominal

# Intrumentos


In [ ]:
from iol_client.constants import Pais


instrumentos = await client_iol.get_instrumentos(pais=Pais.ARG)
instrumentos = pd.DataFrame(data=instrumentos)
instrumentos

# Panel general de un determinado instrumento


In [ ]:
from iol_client.constants import Instrumento


panel = await client_iol.get_paneles(pais=Pais.ARG, instrumento=Instrumento.ARG.BONOS)
panel = pd.DataFrame(data=panel)
panel

# Titulos


In [ ]:
titulo = await client_iol.get_titulo(simbolo="CUAP", mercado=Mercado.BCBA)

In [ ]:
titulo = pd.Series(data=titulo)
titulo

# Opciones de un titulo


In [ ]:
opciones_titulo = await client_iol.get_titulo_opciones(
    mercado=Mercado.BCBA, simbolo="GGAL"
)

In [ ]:
opciones_titulo = pd.DataFrame(data=opciones_titulo)

In [ ]:
cotizacion_opciones_titulo = opciones_titulo["cotizacion"].apply(pd.Series)
cotizacion_opciones_titulo = pd.DataFrame(data=cotizacion_opciones_titulo)

In [ ]:
opciones_titulo = opciones_titulo.drop(columns=["cotizacion"])
opciones_titulo

In [ ]:
cotizacion_opciones_titulo

In [ ]:
cotizacion_opciones_titulo.shape, opciones_titulo.shape

# Cotizacion de un titulo


In [ ]:
cotizacion = await client_iol.get_titulo_cotizacion(
    simbolo="GGAL", mercado=Mercado.BCBA
)

In [ ]:
puntas_cotizacion = cotizacion["puntas"]
puntas_cotizacion = pd.DataFrame(data=puntas_cotizacion)

In [ ]:
puntas_cotizacion

In [ ]:
cotizacion = pd.Series(data=cotizacion)
cotizacion = cotizacion.drop(labels=["puntas"])
cotizacion

# Panel de cotizaciones


In [ ]:
from iol_client.constants import Panel


panel_cotizaciones = await client_iol.get_panel_cotizaciones(
    pais=Pais.ARG,
    instrumento=Instrumento.ARG.BONOS,
    panel=Panel.ARG.BONOS.SOBERANO_ARS_CER,
)

In [ ]:
titulos_panel_cotizaciones = panel_cotizaciones["titulos"]

In [ ]:
def flat_titulo(titulo):
    return {
        **titulo,
        "puntas": None,
        "precioCompra": titulo["puntas"]["precioCompra"],
        "precioVenta": titulo["puntas"]["precioVenta"],
        "cantidadCompra": titulo["puntas"]["cantidadCompra"],
        "cantidadVenta": titulo["puntas"]["cantidadVenta"],
    }

In [ ]:
titulos_flated = [flat_titulo(titulo) for titulo in titulos_panel_cotizaciones]
titulos_flated = pd.DataFrame(data=titulos_flated)
titulos_flated = titulos_flated.drop(columns=["puntas"])
titulos_flated

# Detalle mobile de una cotizacion


In [6]:
from iol_client.constants import Plazo


In [ ]:


cotizacion_detalle_mobile = await client_iol.get_titulo_cotizacion_plazo(
    mercado=Mercado.BCBA, simbolo="GGAL", plazo=Plazo.T0
)
cotizacion_detalle_mobile

In [ ]:
# get all keys of json
keys = [key for key in cotizacion_detalle_mobile.keys()]
keys

In [ ]:
columns = [
    "operableCompra",
    "operableVenta",
    "visible",
    "ultimoPrecio",
    "variacion",
    "apertura",
    "maximo",
    "minimo",
    "fechaHora",
    "tendencia",
    "cierreAnterior",
    "montoOperado",
    "volumenNominal",
    "precioPromedio",
    "moneda",
    "precioAjuste",
    "interesesAbiertos",
    "puntas",
    "cantidadOperaciones",
    "simbolo",
    "pais",
    "mercado",
    "tipo",
    "descripcionTitulo",
    "plazo",
    "laminaMinima",
    "lote",
    "cantidadMinima",
    "puntosVariacion",
]

In [ ]:
columns_without_puntas = [col for col in df.columns if "punta" not in col]

In [ ]:
df_cotizaciones_sin_puntas = pd.Series(data=cotizacion_detalle_mobile)
df_cotizaciones_sin_puntas

In [ ]:
puntas = cotizacion_detalle_mobile["puntas"]
puntas

In [ ]:
cotizaciones_flatted = [{
    **cotizacion_detalle_mobile,
    'puntas': None,
    'cantidadCompra': punta['cantidadCompra'],
    'precioCompra': punta['precioCompra'],
    'precioVenta': punta['precioVenta'],
    'cantidadVenta': punta['cantidadVenta'],
} for punta in puntas ]

df_cotizaciones_flatted = pd.DataFrame(cotizaciones_flatted)
print(df_cotizaciones_flatted)

In [ ]:
df_cotizaciones_flatted.columns

In [ ]:
df_cotizaciones_flatted_out_puntas = df_cotizaciones_flatted.drop(columns=['puntas'])
df_cotizaciones_flatted_out_puntas

In [ ]:
df_cotizaciones_flatted_out_puntas.columns

In [ ]:
array_cotizaciones = df_cotizaciones_flatted_out_puntas.to_records(index=False)
array_cotizaciones

In [ ]:
numpy_array_cotizaciones = df_cotizaciones_flatted_out_puntas.to_numpy().tolist()
numpy_array_cotizaciones

In [7]:

cotizacion_detalle_mobile_t1 = await client_iol.get_titulo_cotizacion_plazo(
    mercado=Mercado.BCBA, simbolo="GGAL", plazo=Plazo.T1
)
cotizacion_detalle_mobile_t1

{'operableCompra': True,
 'operableVenta': False,
 'visible': True,
 'ultimoPrecio': 901.85,
 'variacion': 0.0,
 'apertura': 951.0,
 'maximo': 958.2,
 'minimo': 898.5,
 'fechaHora': datetime.datetime(2023, 7, 28, 3, 0, 1, 377000),
 'tendencia': 'mantiene',
 'cierreAnterior': 901.85,
 'montoOperado': 0.0,
 'volumenNominal': 0,
 'precioPromedio': 0.0,
 'moneda': 'peso_Argentino',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [],
 'cantidadOperaciones': 0,
 'simbolo': 'GGAL',
 'pais': 'argentina',
 'mercado': 'bcba',
 'tipo': 'acciones',
 'descripcionTitulo': 'Grupo Financiero Galicia S.A',
 'plazo': 't1',
 'laminaMinima': 1,
 'lote': 1,
 'cantidadMinima': 1,
 'puntosVariacion': 0.0}